# Our goal is to predict which rating will the user give to the movie
## Importing data

In [ ]:
# Important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/kaggle/input/amazon-top-50-bestselling-books-2009-2019/bestsellers with categories.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum().max()

In [ ]:
# No missing data
df.describe()

In [ ]:
df.shape

# Exploratory data analysis

In [ ]:
# First thing to notice - Name won't really matter here so we will drop this column
df.drop('Name', axis=1, inplace=True)
df.head()

In [ ]:
df['Author'].nunique()

In [ ]:
# There are 248 authors for 550 books - that's around 2 books for one author which is not a lot
# That means that there are a lot of authors that probably have one book
df['Author'].value_counts().median()

In [ ]:
# This means that there are a lot of authors that have only one book in the category
# Now I will calculate how many authors have more than one occurence
authors = df['Author'].value_counts()
authors_data = pd.DataFrame(authors)
authors_data = authors_data.reset_index()
authors_data.columns = ['Author', 'Occurrences']
authors_data[authors_data['Occurrences'] > 1]

In [ ]:
# We see that 118 authors have more than 2 books on the list and 130 authors have only one book
# I will change the author column to reflect that fact
def has_more_than_one_book(author, authors_data):
    if (int(authors_data[authors_data['Author'] == author]['Occurrences']) == 1):
        return 'One Book'
    else:
        return 'More than one book'
    
df['Author'] = df['Author'].apply(lambda x: has_more_than_one_book(x, authors_data))
df.head()

In [ ]:
# Now it's time to see whether this fact affects user rating
df.groupby('Author').describe()['User Rating']

In [ ]:
sns.boxplot(data=df, x='Author', y='User Rating')

In [ ]:
# We can see that only authors that got more than one book on the list got a review lower than 4.0
df['Reviews'].hist(bins=30)

In [ ]:
# Most reviews are within range 0-30000
average_reviews = df.groupby('User Rating').mean()
average_reviews

In [ ]:
plt.figure(figsize=(12, 6))
average_reviews.sort_values(by=['Reviews'])['Reviews'].plot(kind='bar')

In [ ]:
# We can see that there are a ton of reviews for 3.8 user rating and a lot for 3.9 and 4.1 user rating
sns.lineplot(data=df, x='User Rating', y='Reviews')

In [ ]:
plt.figure(figsize=(12, 6))
average_reviews.sort_values(by=['Price'])['Price'].plot(kind='bar')

In [ ]:
# The highest price is for books that have 4.5 rating and lowest for books that have 4.9 rating
# We can see that there are a ton of reviews for 3.8 user rating and a lot for 3.9 and 4.1 user rating
sns.lineplot(data=df, x='User Rating', y='Price')

In [ ]:
plt.figure(figsize=(12, 6))
average_reviews.sort_values(by=['Year'])['Year'].plot(kind='bar', ylim=(2011, 2017))

In [ ]:
# We can see that the oldest books have the rating of 4.2 and the highest around 4.9
sns.lineplot(data=df, x='User Rating', y='Year')

In [ ]:
df['Genre'].value_counts()

In [ ]:
# There is a similar amount of genres there
df.groupby('Genre').mean()['User Rating']

In [ ]:
# Fiction novels tend to have a higher rating than non-fiction ones
sns.boxplot(data=df, x='Genre', y='User Rating')

In [ ]:
# Looking at boxplot we can see that fiction novels have overall much better ratings but they also have
# ratings that are very low compared to non fiction novels

# Training Data

In [ ]:
# I need to create dummy variables before I decide to test my data
data = pd.get_dummies(data=df['Author'], drop_first=True)
df = pd.concat([df, data], axis=1)
df.drop('Author', axis=1, inplace=True)

data = pd.get_dummies(data=df['Genre'], drop_first=True)
df = pd.concat([df, data], axis=1)
df.drop('Genre', axis=1, inplace=True)

df.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('User Rating', axis=1)
y = df['User Rating']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# We will use a few different machine learning algorithms and chech which one of them returns the best results

# Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)
predictions = lm.predict(X_test)

In [ ]:
predictions

In [ ]:
# We can use scatterplot to visualise whether the linear regression was successful in finding user ratings
ax = sns.scatterplot(x=y_test, y=predictions)
ax.set(xlabel='Real values', ylabel='Predictions')
plt.show()

In [ ]:
# We see that linear regression didn't output results that can be easily fitted into a linear function
RMSE_linear_regression = np.sqrt(metrics.mean_squared_error(y_test, predictions))
print('RMSE: ', RMSE_linear_regression)

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dtree = DecisionTreeRegressor(random_state=42)
dtree.fit(X_train, y_train)
predictions = dtree.predict(X_test)

In [ ]:
predictions

In [ ]:
RMSE_decision_tree = np.sqrt(metrics.mean_squared_error(y_test, predictions))
print('RMSE: ', RMSE_decision_tree)

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rtree = RandomForestRegressor(random_state=42)
rtree.fit(X_train, y_train)
predictions = rtree.predict(X_test)

In [ ]:
predictions

In [ ]:
RMSE_random_forest = np.sqrt(metrics.mean_squared_error(y_test, predictions))
print('RMSE: ', RMSE_random_forest)

# Gradient Boosting Tree

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbtree = GradientBoostingRegressor(random_state=42)
gbtree.fit(X_train, y_train)
predictions = gbtree.predict(X_test)

In [ ]:
predictions

In [ ]:
RMSE_boosting_tree = np.sqrt(metrics.mean_squared_error(y_test, predictions))
print('RMSE: ', RMSE_boosting_tree)

# Summary of a machine learning model

In [ ]:
len(predictions)

In [ ]:
RMSE_df = pd.DataFrame(data=[['Linear Regression', RMSE_linear_regression], 
                                ['Decision Tree', RMSE_decision_tree], 
                                ['Random Forest', RMSE_random_forest], 
                                ['Gradient Boosting Tree', RMSE_boosting_tree]],
                         columns=['Model', 'RMSE'])
RMSE_df

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(x='Model', y='RMSE', data=RMSE_df, order=RMSE_df.sort_values('RMSE').Model)
plt.ylim(0.2, 0.3)

### We can see that the decision tree was the worst model in predicting reviews
### Linear regression is the second worst (although it did quite well for a simple machine learning algorithm)
### Random Forest and Gradient Boosting Tree were the best in predicting the user rating

In [ ]:
RMSE_df = RMSE_df.sort_values('RMSE')
RMSE_df.reset_index(drop=True, inplace=True)
RMSE_df